In [2]:
#import cvxpy as cp
import numpy as np
import os
import time
import gurobipy as gp
from gurobipy import GRB
import scipy as sp
from scipy import sparse
from operator import itemgetter
from itertools import groupby

float_formatter = "{:.6f}".format
np.set_printoptions(formatter={'float_kind':float_formatter})

floor = lambda num, precision: ((num*10**precision)//1)/(10**precision)
ceil = lambda num, precision: -((-num*10**precision)//1)/(10**precision)

Reading data

In [3]:
def read_data(problem_name): #some changes needed if only one state, action or observation
    global states_list, obs_list, act_list
    global states, actions, obs
    global obs_states_list, obs_act_list
    global trns_matrices, initial_belief, reward

    file_name = "initial_data/"+problem_name+".txt"

    states_list = np.loadtxt(file_name, dtype = str, max_rows = 2)
    obs_list = np.loadtxt(file_name, dtype = str, skiprows = 3, max_rows = 2)
    act_list = np.loadtxt(file_name, dtype = str, skiprows = 6, max_rows = 2)

    states = len(states_list)
    actions = len(act_list)
    obs = len(obs_list)

    #unequal datatypes
    obs_states_list = []
    for i in range(obs):
        temp = np.loadtxt(file_name, dtype = str, skiprows = 10+i, max_rows = 1)
        obs_states_list.append((temp[0], [np.where(states_list == s)[0][0] for s in temp[1:]]))

    obs_act_list = []
    for i in range(obs):
        temp = np.loadtxt(file_name, dtype = str, skiprows = 12+obs+i, max_rows = 1)
        obs_act_list.append((temp[0], [np.where(act_list == s)[0][0] for s in temp[1:]]))

    trns_data = np.loadtxt(file_name, dtype = float, skiprows = 14+2*obs, max_rows= actions*(states+2)).reshape((actions,states,states*2))
    trns_matrices = []
    for a in range(actions):
        trns_matrices.append(sparse.csr_matrix(trns_data[a]))

    initial_belief_data = np.loadtxt(file_name, dtype = float, skiprows= 14+2*obs+actions*(states+2), max_rows=2).reshape((states,2))
    initial_belief_data[:,0] = -initial_belief_data[:,0]
    initial_belief = (sparse.csr_matrix(initial_belief_data), [])

    reward = np.transpose(np.loadtxt(file_name, dtype = float, skiprows= 14+2*obs+actions*(states+2)+3, max_rows=states).reshape(states,actions))

In [4]:
def read_data_sparse(problem_name):
    global states_list, obs_list, act_list
    global states, actions, obs
    global obs_states_list, obs_act_list
    global trns_matrices, initial_belief, reward

    file_name = "initial_data/"+problem_name+".txt"

    states_list = np.loadtxt(file_name, dtype = str, max_rows = 2)
    obs_list = np.loadtxt(file_name, dtype = str, skiprows = 3, max_rows = 2)
    act_list = np.loadtxt(file_name, dtype = str, skiprows = 6, max_rows = 2)

    states = len(states_list)
    actions = len(act_list)
    obs = len(obs_list)

    #unequal datatypes
    obs_states_list = []
    for i in range(obs):
        temp = np.loadtxt(file_name, dtype = str, skiprows = 10+i, max_rows = 1)
        obs_states_list.append((temp[0], [np.where(states_list == s)[0][0] for s in temp[1:]]))

    obs_act_list = []
    for i in range(obs):
        temp = np.loadtxt(file_name, dtype = str, skiprows = 12+obs+i, max_rows = 1)
        obs_act_list.append((temp[0], [np.where(act_list == s)[0][0] for s in temp[1:]]))

    nr_initial_states = np.loadtxt(file_name, dtype = int, skiprows= 14+2*obs, max_rows=1)
    initial_belief_data = np.loadtxt(file_name, dtype = str, skiprows= 15+2*obs, max_rows=nr_initial_states).reshape(1,3)
    rows = [int(i[0][1:]) for i in initial_belief_data for j in range(2)]
    cols = [j for i in range(nr_initial_states) for j in [0,1]]
    data = [-float(initial_belief_data[i][j+1]) if j%2==0 else float(initial_belief_data[i][j+1]) for i in range(nr_initial_states) for j in range(2)]
    initial_belief = (sparse.csr_matrix((data,(rows,cols)),shape = (states,2)), [])

    nr_rewards = np.loadtxt(file_name, dtype = int, skiprows= 17+nr_initial_states+2*obs, max_rows=1)
    reward_data = np.loadtxt(file_name, dtype = str, skiprows= 18+nr_initial_states+2*obs, max_rows=nr_rewards)
    reward = np.zeros((states, actions))
    for i in range(nr_rewards):
        s = int(reward_data[i][0])
        a = np.where(act_list == reward_data[i][1])[0][0]
        reward[s,a] = reward_data[i][2]
    reward = np.transpose(reward)
    
    nr_trns = np.loadtxt(file_name, dtype = int, skiprows= 19+nr_initial_states+nr_rewards+2*obs, max_rows=1)
    trns_data =  np.loadtxt(file_name, dtype = str, skiprows = 21+nr_initial_states+nr_rewards+2*obs, max_rows= nr_trns)
    trns_data = sorted(trns_data,key=itemgetter(1))
    trns_data = [[k,list(g)] for k, g in (groupby(trns_data, key=itemgetter(1)))]
    trns_matrices = []
    for a in range(actions):
        rows = [int(item[0]) for i in range(2) for item in trns_data[a][1]]
        cols = [2*int(item[2])+i for i in range(2) for item in trns_data[a][1]]
        data = [float(item[3+i]) for i in range(2) for item in trns_data[a][1]]
        trns_matrices.append(sparse.csr_matrix((data,(rows,cols)),shape = (states,states*2)))

Constants

In [241]:
def update_constants():
    global n,m
    
    n = states**2+states
    m = 2*states**2 + 4*states + 2

Precomputation

In [242]:
def precomp():
    global c_vecs
    global get_Am_n, get_c_vec_a_il_jl
    global relevant_states_list

    data, rows, cols = np.zeros(4*states), np.zeros(4*states), np.zeros(4*states)
    for i in range(states):
        rows[4*i:4*i+4] = i
        cols[4*i:4*i+4] = [i, i+states, 2*states, 2*states+1]
        data[4*i:4*i+4] = [-1,1,1,-1]
    Am_small = sparse.csr_matrix((data,(rows,cols)),shape = (states,2*states+2))
    get_Am_n = lambda n: Am_small[:n,[k*states+i for k in range(2) for i in range(n)]+[2*states, 2*states+1]]

    relevant_states_list = [[[]]*states]*actions
    c_vecs = [None] * actions
    for a in range(actions):
        data, rows, cols = [],[],[]
        temp_list = [[]]*states
        trns_rows, trns_cols = trns_matrices[a].nonzero()
        rows += [i for i in trns_rows]
        cols += [j/2 if j%2 == 0 else ((j-1)/2)+states for j in trns_cols]
        data += [-trns_matrices[a][i,j] if j%2 == 0 else trns_matrices[a][i,j] for i, j in zip(trns_rows, trns_cols)]
        for i in range(states):
            if (trns_matrices[a][i].getnnz() > 0):
                rows += [i, i]
                cols += [2*states, 2*states+1]
                data += [1, -1]
                temp_list[i] = [int(j/2) for j in trns_matrices[a][i].nonzero()[1] if j%2==0]
        relevant_states_list[a] = temp_list
        c_vecs[a] = sparse.csr_matrix((data,(rows,cols)),shape = (states,2*states+2))
    get_c_vec_a_il_jl = lambda a, il, jl: c_vecs[a][[i for i in il],:][:,[j+k*states for k in range(2) for j in jl]+[2*states, 2*states+1]]

    for a in range(actions):
        print(f"Action: {act_list[a]}")
        for i in range(states):
            suc_states = relevant_states_list[a][i]
            nr_suc_states = len(suc_states)
            
            if(nr_suc_states > 0):
                transition_model = gp.Model("Model over the uncertain transitions from one state")
                transition_model.params.LogToConsole = 0
                transition_model.params.TimeLimit = 120

                xs = transition_model.addMVar(shape = nr_suc_states, vtype = GRB.CONTINUOUS, name = "xs")
                transition_model.addConstr(np.transpose(get_Am_n(nr_suc_states)) @ xs <= get_c_vec_a_il_jl(a,[i],suc_states), name = "c")

                for j in range(nr_suc_states):
                    obj_vec = np.zeros(nr_suc_states)
                    obj_vec[j] = 1
                    try:
                        transition_model.setObjective(obj_vec @ xs, GRB.MINIMIZE)
                        transition_model.optimize()
                        if transition_model.status == GRB.OPTIMAL:
                            min_data = -np.clip(round(transition_model.objVal, precision), 10**(-precision), 1)
                            # min_data = -transition_model.objVal
                            if min_data < c_vecs[a][i,suc_states[j]]:
                                print(f"Improved lower bound transition P({i},{act_list[a]},{suc_states[j]}): {-c_vecs[a][i,suc_states[j]]} -> {-min_data}")
                                c_vecs[a][i,suc_states[j]] = min_data
                        else:
                            print(f"Not optimal min. State: {i}, State: {suc_states[j]}")
                    except gp.GurobiError as e:
                        print("Error code " + str(e.errno) + ": " + str(e))
                    except AttributeError:
                        print("Encountered an attribute error")

                    try:
                        transition_model.setObjective(obj_vec @ xs, GRB.MAXIMIZE)
                        transition_model.optimize()
                        if transition_model.status == GRB.OPTIMAL:
                            max_data = np.clip(round(transition_model.objVal, precision), 10**(-precision), 1)
                            # max_data = transition_model.objVal
                            if max_data < c_vecs[a][i,states+suc_states[j]]:
                                c_vecs[a][i,states+suc_states[j]] = max_data
                                print(f"Improved upper bound transition P({i},{act_list[a]},{suc_states[j]}): {c_vecs[a][i,states+suc_states[j]]} -> {max_data}")
                        else:
                            print(f"Not optimal max. State: {i}, State: {suc_states[j]}")
                    except gp.GurobiError as e:
                        print("Error code " + str(e.errno) + ": " + str(e))
                    except AttributeError:
                        print("Encountered an attribute error")
                

In [246]:
def precomp_min_max():
    global maximums, minimums
    
    maximums, minimums = [None]*actions, [None]*actions

    for a in range(actions):
        min_data, max_data, rows, cols = [],[],[],[]
        print(f"Action: {act_list[a]}")
        for i in range(states):
            suc_states = relevant_states_list[a][i]
            nr_suc_states = len(suc_states)
            
            if(nr_suc_states > 0):
                transition_model = gp.Model("Model over the uncertain transitions from one state")
                transition_model.params.LogToConsole = 0
                transition_model.params.TimeLimit = 120

                xs = transition_model.addMVar(shape = nr_suc_states, vtype = GRB.CONTINUOUS, name = "xs")
                transition_model.addConstr(np.transpose(get_Am_n(nr_suc_states)) @ xs <= get_c_vec_a_il_jl(a,[i],suc_states), name = "c")
                
                for o in range(obs):
                    obs_suc_states = (set(suc_states) & set(obs_states_list[o][1]))
                    if len(obs_suc_states) > 0:
                        obj_vec = np.array([int(s in obs_suc_states) for s in suc_states])
                        rows += [o]
                        cols += [i]
                        try:
                            transition_model.setObjective(obj_vec @ xs, GRB.MINIMIZE)
                            transition_model.optimize()
                            if transition_model.status == GRB.OPTIMAL:
                                min_data += [np.clip(round(transition_model.objVal, precision), 10**(-precision), 1)]
                            else:
                                print(f"Not optimal min. State: {i}, Rel states: {suc_states}")
                                min_data += [float('inf')]
                        except gp.GurobiError as e:
                            print("Error code " + str(e.errno) + ": " + str(e))
                        except AttributeError:
                            print("Encountered an attribute error")

                        try:
                            transition_model.setObjective(obj_vec @ xs, GRB.MAXIMIZE)
                            transition_model.optimize()
                            if transition_model.status == GRB.OPTIMAL:
                                max_data += [np.clip(round(transition_model.objVal, precision), 10**(-precision), 1)]
                                # max_data += [transition_model.objVal]
                            else:
                                print(f"Not optimal max. State: {i}, Rel states: {suc_states}")
                                max_data += [float('-inf')]
                        except gp.GurobiError as e:
                            print("Error code " + str(e.errno) + ": " + str(e))
                        except AttributeError:
                            print("Encountered an attribute error")
        minimums[a] = sparse.csr_matrix((min_data,(rows,cols)),shape = (obs,states))
        maximums[a] = sparse.csr_matrix((max_data,(rows,cols)),shape = (obs,states))


Precompute transition samples

In [248]:
def precomp_trns_samples(spud, factor):
    global action_trns_samples
    global get_trns_samples

    # def generate_random_samples(c_vec, spud, precision):
    # usefull states and constant states
    action_trns_samples = [None]*actions
    nr_trns_samples = np.ones((actions, states), int)
    for a in range(actions):
        print(f"Action: {act_list[a]}")
        trns_samples = [None]*states
        for i in range(states):
            suc_states = relevant_states_list[a][i]
            nr_suc_states = len(suc_states)

            c_vec = get_c_vec_a_il_jl(a, [i], suc_states)
            constant_states = [j for j in range(nr_suc_states) if (-c_vec[0,j] == c_vec[0,nr_suc_states+j])]
            nz_nc_states = [j for j in range(nr_suc_states) if (-c_vec[0,j] != c_vec[0,nr_suc_states+j])]

            c_rows, c_cols, c_data = [],[],[]
            for j in constant_states:
                c_rows += [0]
                c_cols += [suc_states[j]]
                c_data += [c_vec[0,nr_suc_states+j]]
            nr_nz_nc_states = len(nz_nc_states)
            if nr_nz_nc_states == 0:
                trns_samples[i] = sparse.csr_matrix((c_data,(c_rows,c_cols)),shape = (1,states))
            elif nr_nz_nc_states == 1:
                j = nz_nc_states[0]
                temp = 1 - sum(c_data)
                if -c_vec[0,j] <= temp and temp <= c_vec[0,nr_suc_states+j]:
                    c_rows += [0]
                    c_cols += [suc_states[j]]
                    c_data += temp
                    trns_samples[i] = sparse.csr_matrix((c_data,(c_rows,c_cols)),shape = (1,states))
                else:
                    print(f"BIG ERROR, TRANSITIONS FROM STATE {i} INFEASIBLE")
            else:
                nr_samples = round(sum(round(c_vec[0,nr_suc_states+nz_nc_states[j]] + c_vec[0,nz_nc_states[j]],precision) for j in range(nr_nz_nc_states))*spud)
                nr_samples = max(nr_samples, 1)
                nr_trns_samples[a][i] = int(np.copy(nr_samples))
                nr_samples = factor*nr_samples

                nz_nc_rows, nz_nc_cols, nz_nc_data = [],[],[]
                for sample in range(nr_samples):
                    while True:
                        rows, cols, data = [],[],[]
                        
                        for j in range(nr_nz_nc_states-1):
                            k = nz_nc_states[j]
                            rows += [sample]
                            cols += [suc_states[k]]
                            data += [np.random.uniform(-c_vec[0,k], c_vec[0,nr_suc_states+k])]
                    
                        k = nz_nc_states[-1]
                        temp = 1 - (sum(c_data)+sum(data))
                        if -c_vec[0,k] <= temp and temp <= c_vec[0,nr_suc_states+k]:
                            nz_nc_rows += [sample]*len(constant_states) + rows + [sample]
                            nz_nc_cols += c_cols + cols + [suc_states[k]]
                            nz_nc_data += c_data + data + [temp]
                            break
                trns_samples[i] = sparse.csr_matrix((nz_nc_data,(nz_nc_rows,nz_nc_cols)),shape = (nr_samples,states))
                
        action_trns_samples[a] = trns_samples

    get_trns_samples = lambda a, il: [action_trns_samples[a][i][np.random.randint(0,action_trns_samples[a][i].shape[0])] for i in il]

    return nr_trns_samples

Update belief constraints

In [250]:
def update_belief_constraints(belief, nzb_states):
    global belief_model, ys, t
    global bui_cons, bdc_cons, bdc_A, c_bui_bpd
    
    belief_model = None
    nr_nzb_states = len(nzb_states)
    if nr_nzb_states == 0:
        print("BIG ERROR, ALL ZERO BELIEF STATE")
    else:
        bui = np.transpose(belief[0])
        bdc = belief[1]

        belief_model = gp.Model("Model over an old uncertain belief state")
        belief_model.params.LogToConsole = 0
        belief_model.params.TimeLimit = 120

        ys = belief_model.addMVar(shape = nr_nzb_states, vtype = GRB.CONTINUOUS, name = "ys")
        t = belief_model.addMVar(shape =1, vtype = GRB.CONTINUOUS, name = "t")
        t.lb = 1
        t.ub = 1

        bui_cons = np.array(list(bui[bui.nonzero()].A1)+[1,-1])
        try:
            c_bui_bpd = belief_model.addConstr(np.transpose(get_Am_n(nr_nzb_states)) @ ys <= bui_cons*t, name = "c_bui_bpd")
        except:
            print("Error")

        if len(bdc) > 0:
            rows, cols, data = [],[],[]
            for i in range(len(bdc)):
                # need the intersection with nzb_states
                bdc_nzb_indices = [j for j in range(nr_nzb_states) if (nzb_states[j] in bdc[i][0])]
                nzb_bdc_indices = [j for j in range(len(bdc[i][0])) if (bdc[i][0][j] in nzb_states)]
                rows += [i]*len(bdc_nzb_indices)
                cols += bdc_nzb_indices
                data += [bdc[i][1][j] for j in nzb_bdc_indices]
            bdc_A = sparse.csr_matrix((data,(rows,cols)),shape = (len(bdc), nr_nzb_states))
            bdc_cons = np.array([i[2] for i in bdc])
            c_bdc = belief_model.addConstr(bdc_A @ ys <= bdc_cons*t, name = "c_bdc")
        belief_model.update()

Transition computation

In [252]:
def comp_transition(action, observation, nzb_states, precision):
    global belief_model
    
    min_trns = float('inf')
    max_trns = float('-inf')
    
    try:
        belief_model.setObjective(minimums[action][observation,nzb_states]@ys, GRB.MINIMIZE)
        belief_model.optimize()
        if belief_model.status == GRB.OPTIMAL:
            min_trns = round(belief_model.objVal,precision)
        else:
            print("Not optimal min comp_transition")
    except gp.GurobiError as e:
        print("Error code " + str(e.errno) + ": " + str(e))
    except AttributeError:
        print("Encountered an attribute error")
        
    try:
        belief_model.setObjective(maximums[action][observation,nzb_states]@ys, GRB.MAXIMIZE)
        belief_model.optimize()
        if belief_model.status == GRB.OPTIMAL:
            max_trns = round(belief_model.objVal,precision)
        else:
            print("Not optimal max comp_transition")
    except gp.GurobiError as e:
        print("Error code " + str(e.errno) + ": " + str(e))
    except AttributeError:
        print("Encountered an attribute error")

    return [np.clip(min_trns, 10**(-precision), 1), np.clip(max_trns, 10**(-precision), 1)]


Reward computation

In [253]:
def comp_reward(action, nzb_states, precision):
    global belief_model
    a_reward = reward[action][nzb_states]
    
    min_reward = float('inf')
    max_reward = float('-inf')

    try:
        belief_model.setObjective(a_reward@ys, GRB.MINIMIZE)
        belief_model.optimize()
        if belief_model.status == GRB.OPTIMAL:
            min_reward = round(belief_model.objVal,precision)
        else:
            print("Not optimal min comp_reward")
    except gp.GurobiError as e:
        print("Error code " + str(e.errno) + ": " + str(e))
    except AttributeError:
        print("Encountered an attribute error")
        
    try:
        belief_model.setObjective(a_reward@ys, GRB.MAXIMIZE)
        belief_model.optimize()
        if belief_model.status == GRB.OPTIMAL:
            max_reward = round(belief_model.objVal,precision)
        else:
            print("Not optimal max comp_reward")
    except gp.GurobiError as e:
        print("Error code " + str(e.errno) + ": " + str(e))
    except AttributeError:
        print("Encountered an attribute error")

    return [min_reward, max_reward]

Generate random samples

In [254]:
def generate_belief_samples(nzb_states, nr_samples, precision):
    nr_nzb_states = len(nzb_states)

    constant_states = [j for j in range(nr_nzb_states) if (-bui_cons[j] == bui_cons[nr_nzb_states+j])]
    nz_nc_states = [j for j in range(nr_nzb_states) if (-bui_cons[j] != bui_cons[nr_nzb_states+j])]
    
    c_rows, c_cols, c_data = [],[],[]
    for j in constant_states:
        c_rows += [0]
        c_cols += [nzb_states[j]]
        c_data += [bui_cons[nr_nzb_states+j]]
    
    nr_nz_nc_states = len(nz_nc_states)
    if nr_nz_nc_states == 0:
        return sparse.csr_matrix((c_data,(c_rows,c_cols)),shape = (1,states)), 0
    elif nr_nz_nc_states == 1:
        j = nz_nc_states[0]
        temp = 1 - sum(c_data)
        if -bui_cons[j] <= temp and temp <= bui_cons[nr_nzb_states+j]:
            c_rows += [0]
            c_cols += [nzb_states[j]]
            c_data += [temp]
            return sparse.csr_matrix((c_data,(c_rows,c_cols)),shape = (1,states)), 0
        else:
            print(f"BIG ERROR, CURRENT UNCERTAIN BELIEF STATE INFEASIBLE")
    else:
        nz_nc_rows, nz_nc_cols, nz_nc_data = [],[],[]
        for sample in range(nr_samples):
            while True:
                rows, cols, data = [],[],[]
                
                for j in range(nr_nz_nc_states-1):
                    k = nz_nc_states[j]
                    rows += [sample]
                    cols += [nzb_states[k]]
                    data += [np.random.uniform(-bui_cons[k], bui_cons[nr_nzb_states+k])]
            
                k = nz_nc_states[-1]
                temp = 1 - (sum(c_data)+sum(data))
                if -bui_cons[k] <= temp and temp <= bui_cons[nr_nzb_states+k]:
                    check_vec = np.transpose(sparse.csr_matrix((c_data+data+[temp],(np.zeros(nr_nzb_states),constant_states+nz_nc_states)),shape = (1,nr_nzb_states)))
                    if (bdc_A @ check_vec <= bdc_cons).all:
                        nz_nc_rows += [sample]*len(constant_states) + rows + [sample]
                        nz_nc_cols += c_cols + cols + [nzb_states[k]]
                        nz_nc_data += c_data + data + [temp]
                        break
        print("- - - - - - - - - - Belief samples generated - - - - - - - - - -")
        return sparse.csr_matrix((nz_nc_data,(nz_nc_rows,nz_nc_cols)),shape = (nr_samples,states)), nr_samples

Belief update computation (sampling)

In [255]:
def sampling_bui(j, nzb_states, rel_obs_suc_states, trns_samples, belief_sample):
    return sum([belief_sample[0,nzb_states[i]] * trns_samples[i][0,j] for i in range(len(nzb_states))])/sum([belief_sample[0,nzb_states[i]] * sum([trns_samples[i][0,k] for k in rel_obs_suc_states]) for i in range(len(nzb_states))])

def sampling_bc(l, j, nzb_states, trns_samples, belief_sample):
    return (belief_sample[0,nzb_states[l]] * trns_samples[l][0,j])/sum([belief_sample[0,nzb_states[i]] * trns_samples[i][0,j] for i in range(len(nzb_states))])

def sampling_n(l, nzb_states, rel_obs_suc_states, trns_samples, belief_sample):
    return (belief_sample[0,nzb_states[l]] * sum([trns_samples[l][0,k] for k in rel_obs_suc_states]))/sum([belief_sample[0,nzb_states[i]] * sum([trns_samples[i][0,k] for k in rel_obs_suc_states]) for i in range(len(nzb_states))])


In [256]:
def comp_belief_update_sampling(a, o, nzb_states, rel_obs_suc_states, spud, nr_trns_samples, precision, belief):
    new_bui = None
    new_bdc = []
    tries = 0

    if len(rel_obs_suc_states) == 1:
        cs = rel_obs_suc_states[0]
        new_bui = sparse.csr_matrix(([-1.0,1.0],([cs,cs],[0,1])), shape = (states,2))
    else:
        while True:
            nr_nzb_states = len(nzb_states)
            
            nr_bel_samples = max(round(sum(round(bui_cons[nr_nzb_states+j] + bui_cons[j],precision) for j in range(nr_nzb_states))*spud),1)
            nr_samples = max(max(nr_trns_samples[a][nzb_states]), nr_bel_samples)

            bel_samples, nr_bel_samples = generate_belief_samples(nzb_states, nr_samples, precision)
            trns_samples = [get_trns_samples(a,nzb_states) for r in range(nr_samples)]

            # Belief uncertainty intervals
            nr_ros_states = len(rel_obs_suc_states)
            cols = np.zeros(2*nr_ros_states, dtype = int)
            cols[range(1,2*nr_ros_states, 2)] = 1
            rows, data = np.zeros(2*nr_ros_states), np.zeros(2 * nr_ros_states)
            for k in range(nr_ros_states): #rel_obs_suc_states
                j = rel_obs_suc_states[k]
                bui_results = [sampling_bui(j, nzb_states, rel_obs_suc_states, trns_samples[r], bel_samples[min(r,nr_bel_samples)]) for r in range(nr_samples)]
                rows[2*k], rows[2*k+1] = j, j
                data[2*k] = -np.clip(round(min(bui_results),precision), 10**(-precision), 1)
                data[2*k+1] = np.clip(round(max(bui_results),precision), 10**(-precision), 1)
            new_bui = sparse.csr_matrix((data,(rows,cols)), shape = (states,2))

            # Belief distribution constraints
            for k in range(nr_nzb_states):
                i = nzb_states[k]
                i_suc_states = sorted(set(relevant_states_list[a][i]) & set(obs_states_list[o][1]))

                if len(i_suc_states) > 0:
                    min_coeffs, max_coeffs = [None] * len(i_suc_states), [None] * len(i_suc_states)
                    for l in range(len(i_suc_states)):
                        j = i_suc_states[l]
                        bc_results = [sampling_bc(k, j, nzb_states, trns_samples[r], bel_samples[min(r,nr_bel_samples)]) for r in range(nr_samples)]
                        min_coeffs[l] = np.clip(round(min(bc_results),precision), 10**(-precision), 1)
                        max_coeffs[l] = -np.clip(round(max(bc_results),precision),  10**(-precision), 1)

                    n_results = [sampling_n(k, nzb_states, rel_obs_suc_states, trns_samples[r], bel_samples[min(r,nr_bel_samples)]) for r in range(nr_samples)]
                    min_const = -np.clip(round(min(n_results),precision), 10**(-precision), 1)
                    max_const = np.clip(round(max(n_results),precision), 10**(-precision), 1)

                    new_bdc.append((i_suc_states,max_coeffs,min_const))
                    new_bdc.append((i_suc_states,min_coeffs,max_const))
            
            update_belief_constraints((new_bui, new_bdc), rel_obs_suc_states)
            belief_model.setObjective(0, GRB.MINIMIZE)
            belief_model.optimize()
            if belief_model.status != GRB.INFEASIBLE:
                update_belief_constraints(belief, nzb_states)
                break
            elif tries >= 10:
                raise RuntimeError("Not able to compute a feasible new belief state")
            else:
                tries += 1
    return (new_bui, new_bdc)
    

Belief update computation (partial decoupling, Gurobi)

In [257]:
def decoupling_g_bui(a, o, j, nzb_states, precision):
    global belief_model
    nr_nzb_states = len(nzb_states)

    min_bui = float('inf')
    max_bui = float('-inf')
    
    num = lambda ys: sum([ys[l]*-c_vecs[a][nzb_states[l],j] for l in range(nr_nzb_states)])
    denom = lambda ys: sum([ys[l]*maximums[a][o,nzb_states[l]] for l in range(nr_nzb_states)])
    
    z = belief_model.addVar(vtype = GRB.CONTINUOUS, name = "z")
    zc = belief_model.addConstr(z*denom(ys) == num(ys))
    try:
        belief_model.setObjective(z, GRB.MINIMIZE)
        belief_model.optimize()
        if belief_model.status == GRB.OPTIMAL:
            min_bui = round(belief_model.objVal,precision)
        else:
            print("Not optimal min")
    except gp.GurobiError as e:
        print("Error code " + str(e.errno) + ": " + str(e))
    except AttributeError:
        print("Encountered an attribute error")

    num = lambda ys: sum([ys[l]*c_vecs[a][nzb_states[l],states+j] for l in range(nr_nzb_states)])
    denom = lambda ys: sum([ys[l]*minimums[a][o,nzb_states[l]] for l in range(nr_nzb_states)])
    
    belief_model.remove(zc)
    zc = belief_model.addConstr(z*denom(ys) == num(ys))
    try:
        belief_model.setObjective(z, GRB.MAXIMIZE)
        belief_model.optimize()
        if belief_model.status == GRB.OPTIMAL:
            max_bui = round(belief_model.objVal,precision)
        else:
            print("Not optimal max")
    except gp.GurobiError as e:
        print("Error code " + str(e.errno) + ": " + str(e))
    except AttributeError:
        print("Encountered an attribute error")

    belief_model.remove(zc)
    belief_model.remove(z)

    return -np.clip(min_bui, 10**(-precision), 1), np.clip(max_bui, 10**(-precision), 1)

In [258]:
def decoupling_g_bc(a, o, k, j, nzb_states, precision):
    global belief_model
    nr_nzb_states = len(nzb_states)
    i = nzb_states[k]

    min_bc = float('inf')
    max_bc = float('-inf')
    
    num = lambda ys: ys[k]*-c_vecs[a][i,j]
    denom = lambda ys: sum([ys[l]*c_vecs[a][nzb_states[l],states+j] for l in range(nr_nzb_states)])
    
    z = belief_model.addVar(vtype = GRB.CONTINUOUS, name = "z")
    zc = belief_model.addConstr(z*denom(ys) == num(ys))
    try:
        belief_model.setObjective(z, GRB.MINIMIZE)
        belief_model.optimize()
        if belief_model.status == GRB.OPTIMAL:
            min_bc = round(belief_model.objVal,precision)
        else:
            print("Not optimal min")
    except gp.GurobiError as e:
        print("Error code " + str(e.errno) + ": " + str(e))
    except AttributeError:
        print("Encountered an attribute error")

    num = lambda ys: ys[k]*c_vecs[a][i,states+j]
    denom = lambda ys: sum([ys[l]*-c_vecs[a][nzb_states[l],j] for l in range(nr_nzb_states)])
    
    belief_model.remove(zc)
    zc = belief_model.addConstr(z*denom(ys) == num(ys))
    try:
        belief_model.setObjective(z, GRB.MAXIMIZE)
        belief_model.optimize()
        if belief_model.status == GRB.OPTIMAL:
            max_bc = round(belief_model.objVal,precision)
        else:
            print("Not optimal max")
    except gp.GurobiError as e:
        print("Error code " + str(e.errno) + ": " + str(e))
    except AttributeError:
        print("Encountered an attribute error")
    
    belief_model.remove(zc)
    belief_model.remove(z)
    
    return np.clip(min_bc, 10**(-precision), 1), -np.clip(max_bc, 10**(-precision), 1)

In [259]:
def decoupling_g_n(a, o, k, nzb_states, precision):
    global belief_model
    nr_nzb_states = len(nzb_states)
    i = nzb_states[k]

    min_n = float('inf')
    max_n = float('-inf')
    
    num = lambda ys: ys[k]*minimums[a][o,i]
    denom = lambda ys: sum([ys[l]*maximums[a][o,nzb_states[l]] for l in range(nr_nzb_states)])
    
    z = belief_model.addVar(vtype = GRB.CONTINUOUS, name = "z")
    zc = belief_model.addConstr(z*denom(ys) == num(ys))
    try:
        belief_model.setObjective(z, GRB.MINIMIZE)
        belief_model.optimize()
        if belief_model.status == GRB.OPTIMAL:
            min_n = round(belief_model.objVal,precision)
        else:
            print("Not optimal min")
    except gp.GurobiError as e:
        print("Error code " + str(e.errno) + ": " + str(e))
    except AttributeError:
        print("Encountered an attribute error")

    num = lambda ys: ys[k]*maximums[a][o,i]
    denom = lambda ys: sum([ys[l]*minimums[a][o,nzb_states[l]] for l in range(nr_nzb_states)])
    
    belief_model.remove(zc)
    zc = belief_model.addConstr(z*denom(ys) == num(ys))
    try:
        belief_model.setObjective(z, GRB.MAXIMIZE)
        belief_model.optimize()
        if belief_model.status == GRB.OPTIMAL:
            max_n = round(belief_model.objVal,precision)
        else:
            print("Not optimal max")
    except gp.GurobiError as e:
        print("Error code " + str(e.errno) + ": " + str(e))
    except AttributeError:
        print("Encountered an attribute error")

    belief_model.remove(zc)
    belief_model.remove(z)
    
    return -np.clip(min_n, 10**(-precision), 1), np.clip(max_n, 10**(-precision), 1)

In [260]:
def comp_belief_update_decoupling_gurobi(a, o, nzb_states, rel_suc_states, rel_obs_suc_states, precision):
    global belief_model
    new_bui = None
    new_bdc = []
    
    if len(rel_obs_suc_states) == 1:
        cs = rel_obs_suc_states[0]
        new_bui = sparse.csr_matrix(([-1.0,1.0],([cs,cs],[0,1])), shape = (states,2))
    else:
        belief_model.params.NonConvex = 2

        nr_nzb_states = len(nzb_states)
        
        # Belief uncertainty intervals
        nr_ros_states = len(rel_obs_suc_states)
        cols = np.zeros(2*nr_ros_states, dtype = int)
        cols[range(1,2*nr_ros_states, 2)] = 1
        rows, data = np.zeros(2*nr_ros_states), np.zeros(2 * nr_ros_states)
        for k in range(nr_ros_states): #rel_obs_suc_states
            j = rel_obs_suc_states[k]
            rows[2*k], rows[2*k+1] = j, j
            data[2*k], data[2*k+1] = decoupling_g_bui(a, o, j, nzb_states, precision)
        new_bui = sparse.csr_matrix((data,(rows,cols)), shape = (states,2))

        # Belief distribution constraints
        mbs_states = []
        for i in rel_obs_suc_states:
            if rel_suc_states.count(i) > 1:
                mbs_states += [i]
        for k in range(nr_nzb_states):
            i = nzb_states[k]
            i_suc_states = sorted(set(relevant_states_list[a][i]) & set(obs_states_list[o][1]))
            
            if len(i_suc_states) > 0:
                min_coeffs, max_coeffs = [1] * len(i_suc_states), [-1] * len(i_suc_states)
                for l in range(len(i_suc_states)):
                    j = i_suc_states[l]
                    if j in mbs_states:
                        min_coeffs[l], max_coeffs[l] = decoupling_g_bc(a, o, k, j, nzb_states, precision)

                min_const, max_const = decoupling_g_n(a, o, k, nzb_states, precision)

                new_bdc.append((i_suc_states,max_coeffs,min_const))
                new_bdc.append((i_suc_states,min_coeffs,max_const))
        
        belief_model.params.NonConvex = -1
            
    return (new_bui, new_bdc)

Belief update computation (partial decoupling, Charnes-Cooper)

In [261]:
def decoupling_cc_bui(a, o, j, nzb_states, precision):
    global belief_model, t
    nr_nzb_states = len(nzb_states)
    
    min_bui = float('inf')
    max_bui = float('-inf')
    
    num = lambda ys: sum([ys[l]*-c_vecs[a][nzb_states[l],j] for l in range(nr_nzb_states)])
    denom = lambda ys: sum([ys[l]*maximums[a][o,nzb_states[l]] for l in range(nr_nzb_states)])

    zc = belief_model.addConstr(denom(ys) == 1)
    t.lb = 0
    t.ub = float('inf')

    try:
        belief_model.setObjective(num(ys), GRB.MINIMIZE)
        belief_model.optimize()
        if belief_model.status == GRB.OPTIMAL:
            opt_vars = belief_model.getAttr("X", belief_model.getVars())
            min_bui = round(num(opt_vars)/denom(opt_vars),precision)
        else:
            print(f"Not optimal min bui. a: {act_list[a]}, o: {obs_list[o]}, j: {states_list[j]}")
    except gp.GurobiError as e:
        print("Error code " + str(e.errno) + ": " + str(e))
    except AttributeError:
        print("Encountered an attribute error")
    
    num = lambda ys: sum([ys[l]*c_vecs[a][nzb_states[l],states+j] for l in range(nr_nzb_states)])
    denom = lambda ys: sum([ys[l]*minimums[a][o,nzb_states[l]] for l in range(nr_nzb_states)])
    
    belief_model.remove(zc)
    zc = belief_model.addConstr(denom(ys) == 1)
    try:
        belief_model.setObjective(num(ys), GRB.MAXIMIZE)
        belief_model.optimize()
        if belief_model.status == GRB.OPTIMAL:
            opt_vars = belief_model.getAttr("X", belief_model.getVars())
            max_bui = round(num(opt_vars)/denom(opt_vars),precision)
        else:
            print("Not optimal max bui")
    except gp.GurobiError as e:
        print("Error code " + str(e.errno) + ": " + str(e))
    except AttributeError:
        print("Encountered an attribute error")
    
    belief_model.remove(zc)
    t.lb = 1
    t.ub = 1
    
    return -np.clip(min_bui, 10**(-precision), 1), np.clip(max_bui, 10**(-precision), 1)

In [262]:
def decoupling_cc_bc(a, o, k, j, nzb_states, precision):
    global belief_model
    nr_nzb_states = len(nzb_states)
    i = nzb_states[k]
    
    min_bc = float('inf')
    max_bc = float('-inf')
    
    num = lambda ys: ys[k]*-c_vecs[a][i,j]
    denom = lambda ys: sum([ys[l]*c_vecs[a][nzb_states[l],states+j] for l in range(nr_nzb_states)])
    
    zc = belief_model.addConstr(denom(ys) == 1)
    t.lb = 0
    t.ub = float('inf')
    
    try:
        belief_model.setObjective(num(ys), GRB.MINIMIZE)
        belief_model.optimize()
        if belief_model.status == GRB.OPTIMAL:
            opt_vars = belief_model.getAttr("X", belief_model.getVars())
            min_bc = round(num(opt_vars)/denom(opt_vars),precision)
        else:
            print("Not optimal min bc")
    except gp.GurobiError as e:
        print("Error code " + str(e.errno) + ": " + str(e))
    except AttributeError:
        print("Encountered an attribute error")

    num = lambda ys: ys[k]*c_vecs[a][i,states+j]
    denom = lambda ys: sum([ys[l]*-c_vecs[a][nzb_states[l],j] for l in range(nr_nzb_states)])
    
    belief_model.remove(zc)
    zc = belief_model.addConstr(denom(ys) == 1)
    try:
        belief_model.setObjective(num(ys), GRB.MAXIMIZE)
        belief_model.optimize()
        if belief_model.status == GRB.OPTIMAL:
            opt_vars = belief_model.getAttr("X", belief_model.getVars())
            max_bc = round(num(opt_vars)/denom(opt_vars),precision)
        else:
            print("Not optimal max bc")
    except gp.GurobiError as e:
        print("Error code " + str(e.errno) + ": " + str(e))
    except AttributeError:
        print("Encountered an attribute error")
    
    belief_model.remove(zc)
    t.lb = 1
    t.ub = 1
    
    return np.clip(min_bc, 10**(-precision), 1), -np.clip(max_bc, 10**(-precision), 1)

In [263]:
def decoupling_cc_n(a, o, k, nzb_states, precision):
    global belief_model
    nr_nzb_states = len(nzb_states)
    i = nzb_states[k]

    min_n = float('inf')
    max_n = float('-inf')
    
    num = lambda ys: ys[k]*minimums[a][o,i]
    denom = lambda ys: sum([ys[l]*maximums[a][o,nzb_states[l]] for l in range(nr_nzb_states)])
    
    zc = belief_model.addConstr(denom(ys) == 1)
    t.lb = 0
    t.ub = float('inf')
    
    try:
        belief_model.setObjective(num(ys), GRB.MINIMIZE)
        belief_model.optimize()
        if belief_model.status == GRB.OPTIMAL:
            opt_vars = belief_model.getAttr("X", belief_model.getVars())
            min_n = round(num(opt_vars)/denom(opt_vars),precision)
        else:
            print("Not optimal min n")
    except gp.GurobiError as e:
        print("Error code " + str(e.errno) + ": " + str(e))
    except AttributeError:
        print("Encountered an attribute error")

    num = lambda ys: ys[k]*maximums[a][o,i]
    denom = lambda ys: sum([ys[l]*minimums[a][o,nzb_states[l]] for l in range(nr_nzb_states)])
    
    belief_model.remove(zc)
    zc = belief_model.addConstr(denom(ys) == 1)
    try:
        belief_model.setObjective(num(ys), GRB.MAXIMIZE)
        belief_model.optimize()
        if belief_model.status == GRB.OPTIMAL:
            opt_vars = belief_model.getAttr("X", belief_model.getVars())
            max_n = round(num(opt_vars)/denom(opt_vars),precision)
        else:
            print("Not optimal max n")
    except gp.GurobiError as e:
        print("Error code " + str(e.errno) + ": " + str(e))
    except AttributeError:
        print("Encountered an attribute error")

    belief_model.remove(zc)
    t.lb = 1
    t.ub = 1
    
    return -np.clip(min_n, 10**(-precision), 1), np.clip(max_n, 10**(-precision), 1)

In [264]:
def comp_belief_update_decoupling_charnes_cooper(a, o, nzb_states, rel_suc_states, rel_obs_suc_states, precision):
    new_bui = None
    new_bdc = []
    
    if len(rel_obs_suc_states) == 1:
        cs = rel_obs_suc_states[0]
        new_bui = sparse.csr_matrix(([-1.0,1.0],([cs,cs],[0,1])), shape = (states,2))
    else:
        nr_nzb_states = len(nzb_states)
        
        # Belief uncertainty intervals
        nr_ros_states = len(rel_obs_suc_states)
        cols = np.zeros(2*nr_ros_states, dtype = int)
        cols[range(1,2*nr_ros_states, 2)] = 1
        rows, data = np.zeros(2*nr_ros_states), np.zeros(2 * nr_ros_states)
        for k in range(nr_ros_states): #rel_obs_suc_states
            j = rel_obs_suc_states[k]
            rows[2*k], rows[2*k+1] = j, j
            data[2*k], data[2*k+1] = decoupling_cc_bui(a, o, j, nzb_states, precision)
        new_bui = sparse.csr_matrix((data,(rows,cols)), shape = (states,2))

        # Belief distribution constraints
        mbs_states = []
        for i in rel_obs_suc_states:
            if rel_suc_states.count(i) > 1:
                mbs_states += [i]
        for k in range(nr_nzb_states):
            i = nzb_states[k]
            i_suc_states = sorted(set(relevant_states_list[a][i]) & set(obs_states_list[o][1]))

            if len(i_suc_states) > 0:
                min_coeffs, max_coeffs = [1] * len(i_suc_states), [-1] * len(i_suc_states)
                for l in range(len(i_suc_states)):
                    j = i_suc_states[l]
                    if j in mbs_states:
                        min_coeffs[l], max_coeffs[l] = decoupling_cc_bc(a, o, k, j, nzb_states, precision)

                min_const, max_const = decoupling_cc_n(a, o, k, nzb_states, precision)

                new_bdc.append((i_suc_states,max_coeffs,min_const))
                new_bdc.append((i_suc_states,min_coeffs,max_const))

    return (new_bui, new_bdc)

Main loop (iterate)

In [266]:
def iterate(method, spud, factor, nr_trns_samples, problem_name, experiment, precision, minutes):
    beliefs = [initial_belief]
    rewards = []
    transitions = []
    times = []

    # uncertain belief states counter
    ubsc = 0

    init_obs =  next(j for j in range(obs) if (next(i for i in range(states) if beliefs[ubsc][0][i,0] != 0 or beliefs[ubsc][0][i,1] != 0)) in obs_states_list[j][1])
    b_info = [[0, -1, init_obs]]
    t_info = []
    r_info = []

    while ubsc < len(beliefs):# and ubsc <= 21:
        try:
            # current observation
            co = b_info[ubsc][2]
            # non-zero belief states
            nzb_states = beliefs[ubsc][0][:,1].nonzero()[0]

            update_belief_constraints(beliefs[ubsc], nzb_states)
            for a in obs_act_list[co][1]:
                rel_suc_states = [j for i in nzb_states for j in relevant_states_list[a][i]]

                for o in range(obs):
                    rel_obs_suc_states = sorted(set(rel_suc_states) & set(obs_states_list[o][1]))

                    if len(rel_obs_suc_states) >= 1:
                        new_belief = None
                        if method == "sampling":
                            new_belief = comp_belief_update_sampling(a, o, nzb_states, rel_obs_suc_states, spud, nr_trns_samples, precision, beliefs[ubsc])
                        elif method == "pd_gurobi":
                            new_belief = comp_belief_update_decoupling_gurobi(a, o, nzb_states, rel_suc_states, rel_obs_suc_states, precision)
                        else:
                            new_belief = comp_belief_update_decoupling_charnes_cooper(a, o, nzb_states, rel_suc_states, rel_obs_suc_states, precision)
                        transitions.append(comp_transition(a,o, nzb_states, precision))
                        
                        beliefs.append(new_belief)
                        b_info.append([ubsc, a, o])
                        t_info.append([ubsc, a, o, len(beliefs)-1])
                    if((time.time() - start_time) > minutes*60):
                        break
                
                rewards.append(comp_reward(a, nzb_states, precision))
                r_info.append([ubsc, a])
                if((time.time() - start_time) > minutes*60):
                    break
            ubsc += 1
            
            iter_time = time.time() - start_time
            times.append([problem_name, ubsc, method, spud, iter_time])
            if(ubsc % 10 == 0):
                print("Problem: %s\t Method: %s\t States explored: %i\t Sample spacing: %f\t Duration = %f seconds" % (prob,method, ubsc, spud, iter_time))
            
            if(iter_time > minutes*60):
                print(f"Timeout ({minutes} minutes)")
                break
            #else:
            #    print("Next uncertain belief state")
        except Exception as exc:
            print("Caught exception:", exc)
            break
    
    dir_name = "data%i/%s_%s_%s_f%s/" % (experiment,problem_name,method,str(spud),str(factor))
    os.makedirs(os.path.dirname(dir_name+"beliefs.txt"), exist_ok=True)
    with open(dir_name+"beliefs.txt", "w", newline="") as outfile:
        for row in beliefs:
            bel_rows = row[0][:,0].nonzero()[0]
            for br in bel_rows:
                outfile.write(f"{states_list[br]} {round(row[0][br,0], precision)} {round(row[0][br,1],precision)} ")
            outfile.write(f"\n{len(row[1])}\n")
            for item in row[1]:
                outfile.write(" ".join(map(str,item))+"\n")
            outfile.write("\n")
    
    with open(dir_name+"b_info.txt", "w", newline="") as outfile:
        for item in b_info:
            outfile.write(" ".join(map(str,item))+"\n")

    with open(dir_name+"transitions.txt", "w", newline="") as outfile:
        for item in transitions:
            outfile.write(" ".join(map(str,item))+"\n")
    
    with open(dir_name+"t_info.txt", "w", newline="") as outfile:
        for item in t_info:
            outfile.write(" ".join(map(str,item))+"\n")

    with open(dir_name+"rewards.txt", "w", newline="") as outfile:
        for item in rewards:
            outfile.write(" ".join(map(str,item))+"\n")
    
    with open(dir_name+"r_info.txt", "w", newline="") as outfile:
        for item in r_info:
            outfile.write(" ".join(map(str,item))+"\n")
    
    with open(dir_name+"times.txt", "w", newline="") as outfile:
        for item in times:
            outfile.write(" ".join(map(str,item))+"\n")

    return len(beliefs)

In [ ]:
experiment = 3
start_time = None
problems = ["cheese_maze_u1", "cheese_maze_u2", "cheese_maze_u3", "cheese_maze_u4", "cheese_maze_u5"]
methods = ["pd_gurobi", "pd_cc", "sampling"]
spuds = [100, 1000]
factor = [2,5]
precision = 6
minutes = 5

for prob in problems:
    for method in methods:
        if method == "sampling":
            for spud in spuds:
                for fac in factor:
                    start_time = time.time()
                    if prob == "aircraft":
                        read_data_sparse(prob)
                    else:
                        read_data(prob)
                    print("Read data")
                    update_constants()
                    print("Updated constants")
                    precomp()
                    print("Completed precomp")
                    precomp_min_max()
                    print("Completed min_max")

                    nr_trns_samples = precomp_trns_samples(spud, fac)
                    print("Completed precomp trns samples")
                    explored = iterate("sampling", spud, fac, nr_trns_samples, prob, experiment, precision, minutes)
                    print("Finished problem: %s\t Method: %s\t States found: %i\t Sample spacing: %i" % (prob,"sampling", explored, spud))
        else:
            start_time = time.time()
            if prob == "aircraft":
                read_data_sparse(prob)
            else:
                read_data(prob)
            print("Read data")
            update_constants()
            print("Updated constants")
            precomp()
            print("Completed precomp")
            precomp_min_max()
            print("Completed min_max")

            explored = iterate(method, 0, 0, [], prob, experiment, precision, minutes)
            print("Finished problem: %s\t Method: %s\t States found: %i" % (prob,method, explored))
